In [187]:
# Imports
import networkx as nx
import numpy as np
import cvxpy as cp

In [188]:
# Creating the test graph
G = nx.MultiDiGraph()
# G.add_edge(1, 2, weight=1, isNew=False)
G.add_edge(1, 2, weight=np.nan, noWay=0, isClosed=0, length=20, speed=18, maxSpeed=20)
G.add_edge(1, 3, weight=np.nan, noWay=0, isClosed=0, length=20, speed=15, maxSpeed=20)
G.add_edge(2, 4, weight=np.nan, noWay=0, isClosed=0, length=20, speed=10, maxSpeed=20)
G.add_edge(3, 4, weight=np.nan, noWay=0, isClosed=0, length=20, speed=7, maxSpeed=20)
G.add_edge(4, 5, weight=np.nan, noWay=0, isClosed=1, length=20, speed=5, maxSpeed=20)
desiredPath = [1, 3, 4]

In [189]:
# # Other  direction, for testing
#G.add_edge(2, 1, weight=np.nan, noWay=0, isClosed=0, length=20, speed=18, maxSpeed=20)
# G.add_edge(3, 1, weight=np.nan, noWay=0, isClosed=0, length=20, speed=15, maxSpeed=20)
# G.add_edge(4, 2, weight=np.nan, noWay=0, isClosed=0, length=20, speed=10, maxSpeed=20)
# G.add_edge(4, 3, weight=np.nan, noWay=0, isClosed=0, length=20, speed=7, maxSpeed=20)
# G.add_edge(5, 4, weight=np.nan, noWay=0, isClosed=1, length=20, speed=5, maxSpeed=20)

In [190]:
# Helper Functions

# def calculateWeight(data):
#     inf = 1000000
    
#     if data['noWay'] == 1:
#         weight = inf
#     elif data['isClosed'] == 1:
#         weight = inf
#     else:
#         weight = getInverse(data['speed']) * data['length']
        
#     return weight

def calculateWeight(data):
    inf = 1000000

    weight = getInverse(data['speed']) * data['length'] + inf * data['noWay'] + inf * data['isClosed']
        
    return weight

def updateEdgeWeight(graph, source, target, data):
    graph[source][target][0]['weight'] = calculateWeight(data)

def updateGraphWeights(graph):
    for (i, j, data) in graph.edges(data=True):
        updateEdgeWeight(graph, i, j, data)
    
def getPathWeight(path, graph):
    weights = 0
    for i in range(len(path) - 1):
        j = i + 1
        source = path[i]
        target = path[j]
        weights += graph[source][target][0]['weight']
        
    return weights
        
def getInverse(speed):
    return 1/speed

def addReverseEdges(graph):
    for (i, j) in graph.edges():
        if (j, i) not in graph.edges():
            graph.add_edge(j, i, weight=np.nan, noWay=1, isClosed=0, length=0, speed=1, maxSpeed=1)
            updateEdgeWeight(graph, j, i, graph[j][i][0])
    
def cleanGraphAttributes(graph):
    for (i, j) in graph.edges():
        if graph[i][j][0]['isClosed'] == 1:
            graph[i][j][0]['length'] = 0
    

In [191]:
cleanGraphAttributes(G)
updateGraphWeights(G)

In [205]:
def inverseShortestPath(original_graph, desiredPath):
    # auxiliary variables
    
    graph = original_graph.copy()
    addReverseEdges(graph)

    inf = 1000000
    epsilon = 0.001
        
    n = len(graph.nodes())

    m = len(graph.edges())
    
    source = desiredPath[0]
    target = desiredPath[len(desiredPath) - 1]

    noWay = []
    areClosed = []
    inverseSpeeds = []
    inverseMaxSpeeds = []
    lengths = []

    edges = []
    edgeIndex = {}
    for (i, j, data) in graph.edges(data=True):
        # Edges Index
        edgeIndex[i,j] = len(edges)
        # Add edge
        edges.append([i, j])
        
        # Data:
        # (not) is there an edge:
        noWay.append(data['noWay'])
        # closed or not
        areClosed.append(data['isClosed'])
        #get the inverse of the speed
        inverseSpeeds.append(getInverse(data['speed']))
        #get the inverse of the max speed
        inverseMaxSpeeds.append(getInverse(data['maxSpeed']))
        
        lengths.append(data['length'])    
        
        
    nodeIndex = {}
    nodes = []
    for n in graph.nodes:
        # Node Index
        nodeIndex[n] = len(nodes)
        # Add Node
        nodes.append(n)
        
        
    # Ax = b
    A = np.zeros([len(nodes), len(edges)])
    b = np.zeros(len(nodes))

    for i in range(len(nodes)):
        for neighbour in graph.adj[nodes[i]]:
            # Filling A
            j = edgeIndex[nodes[i], neighbour]
            A[i,j] = 1
            if (neighbour, nodes[i]) in edgeIndex:
                j = edgeIndex[neighbour, nodes[i]]
                A[i,j] =-1
            
        # Filling b
        if nodes[i] == source:
            b[i] = 1
        if nodes[i] == target:
            b[i] = -1
            
            
    # optimal x
    path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    xstar = np.zeros(len(edges))
    for p in range(len(path)-1):
        j = edgeIndex[path[p], path[p+1]]
        xstar[j] = 1

    # desired x
    xzero = np.zeros(len(edges))
    for p in range(len(desiredPath)-1):
        j = edgeIndex[desiredPath[p], desiredPath[p+1]]
        xzero[j] = 1
        


    # LP (ISP):
    
    noWay_original = np.asarray(noWay)
    areClosed_original = np.asarray(areClosed)
    inverseSpeeds_original = np.asarray(inverseSpeeds)
    inverseMaxSpeeds_original = np.asarray(inverseMaxSpeeds)
    
    #w_ = cp.Variable(len(w_original))
    pi_ = cp.Variable(len(nodes)) #(2d)
    lambda_ = cp.Variable(len(edges)) #(2d)
    
    # more vars
    noWay_ = cp.Variable(len(noWay_original), boolean=True)
    areClosed_ = cp.Variable(len(areClosed_original), boolean=True)
    inverseSpeeds_ = cp.Variable(len(inverseSpeeds_original))
    #inverseMaxSpeeds_ = cp.Variable(len(inverseMaxSpeeds_original))
    
    # weights for L1norm (not-on-path-penalty) # Why do we use this???????
    # penalty = np.array([1]*len(w_original))
    # for i in range(len(nodes)):
    #     if nodes[i] not in desiredPath[1:-1]:
    #         penalty[i] = 10
        
        
        
    constraints = []
    
    # (2b) & (2c) 
    for j in range(len(edges)):
        d_j = inverseSpeeds_[j] * lengths[j]  + inf * noWay_[j] + inf * areClosed_[j]
        
        if xzero[j] == 1:
        # sum_i a_ij * pi_i = d_j,              for all j in desired path (2b)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) == d_j )
        else:
        # sum_i a_ij * pi_i + lambda_j = d_j,   for all j not in desired path (2c)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) + lambda_[j] == d_j )
        
        
    #(2e)
    for j in range(len(edges)):
        if xzero[j] == 0:
            constraints.append( lambda_[j] >= 0 )
            
            
    # sum_k l_ik = 1, for all i (5g, from NISP)
    # for i in range(len(varnodes)):
    #     idx = len(allowedAreaTypes) * i
    #     constraints.append( cp.sum(l_[idx:idx+len(allowedAreaTypes)]) == 1 )
            
    # w>=0
    #constraints.append(w_ >= epsilon)
    
    for j in range(len(edges)):
        if xzero[j] == 0:
            #if not in desired path, do not change the data 
            #constraints.append( w_[j] == w_original[j] )
            constraints.append( noWay_[j] == noWay_original[j] )
            constraints.append( areClosed_[j] == areClosed_original[j] )
            constraints.append( inverseSpeeds_[j] == inverseSpeeds_original[j] )
            #constraints.append( inverseMaxSpeeds_[j] == inverseMaxSpeeds_original[j] )
        
        
        # # inverseSpeed is at least the max speed; ie speed is at most max speed.
        else:
            constraints.append(  inverseSpeeds_[j] >= inverseMaxSpeeds_original[j] )
    
            
            
    
    # # new edges:
    # for j in range(len(edges)):
    #     d_j = w_[j]
    #     if noWay_original[j] == 0:
    #         #if not in desired path, do not change the weight 
    #         constraints.append( w_[j] == w_original[j] )
    
    
    
    cost1 = cp.norm1(inverseSpeeds_ - inverseSpeeds_original)
    cost2 = cp.norm1(noWay_ - noWay_original)
    cost3 = cp.norm1(areClosed_ - areClosed_original)
            
            
    # Cost funnction
    cost = cost1 + cost2 + cost3 # ||w' - w||1 (2a)
    
    #Check why the penalty??
    #cost = cp.norm1(cp.multiply(w_ - w_original, penalty))
        
    # Forming the problem
    prob = cp.Problem(cp.Minimize(cost), constraints)
    
    # Solve the problem
    #prob.solve(verbose=True)#Detailed
    prob.solve()  # Returns the optimal value.
    #prob.solve(solver=cp.GUROBI) # using gurobi
    print("\nThe optimal value is", prob.value)
    
    
    print('original speedInv: ', inverseSpeeds_original)
    print('optimal speedInv: ', inverseSpeeds_.value)
    print('\n')
    print('original noWay: ', noWay_original)
    print('optimal noWay: ', noWay_.value)
    print('\n')
    print('original areClosed: ', areClosed_original)
    print('optimal areClosed: ', areClosed_.value)
    
    
    newGraph = nx.MultiDiGraph()
    
    for (i, j), index in edgeIndex.items():
        newGraph.add_edge(
                            i, j, 
                            #weight = w_.value[index], 
                            weight = np.nan, 
                            noWay = noWay_.value[index], 
                            isClosed = areClosed_.value[index], 
                            length=graph[i][j][0]['length'], 
                            speed = getInverse(inverseSpeeds_.value[index]), 
                            maxSpeed=graph[i][j][0]['maxSpeed']
                        )
        
    updateGraphWeights(newGraph)
        
        
    sp = nx.shortest_path(newGraph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    
    desiredPathWeight = getPathWeight(desiredPath, newGraph)
    optimalPathWeight = getPathWeight(sp, newGraph)
    
    print('\n')
    
    if desiredPathWeight == optimalPathWeight:
        print('The desired Path is equal to the Shortest Path')
    elif desiredPathWeight < optimalPathWeight:
        print('The desired Path is better than the Shortest Path')
    elif desiredPathWeight > optimalPathWeight:
        print('The desired Path is worse than the Shortest Path')
    
    
    print('Optimal Path Weight = ', optimalPathWeight)
    print('The path is: ', sp)
    print('numbers after decimal point: ', len(str(optimalPathWeight).replace('.','')) - 1)
    print('\n')
    print('Desired Path Weight = ', desiredPathWeight)
    print('The path is: ', desiredPath)
    print('numbers after decimal point: ', len(str(desiredPathWeight).replace('.','')) - 1)
    
    
        
    return newGraph


In [206]:
# Solving using the function

new_graph = inverseShortestPath(G, desiredPath)


The optimal value is 0.05396825396825396
original speedInv:  [0.05555556 0.06666667 0.1        1.         0.14285714 1.
 0.2        1.         1.         1.        ]
optimal speedInv:  [0.05555556 0.05       0.1        1.         0.10555556 1.
 0.2        1.         1.         1.        ]


original noWay:  [0 0 0 1 0 1 0 1 1 1]
optimal noWay:  [0. 0. 0. 1. 0. 1. 0. 1. 1. 1.]


original areClosed:  [0 0 0 0 0 0 1 0 0 0]
optimal areClosed:  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


The desired Path is worse than the Shortest Path
Optimal Path Weight =  3.111111111111111
The path is:  [1, 2, 4]
numbers after decimal point:  15


Desired Path Weight =  3.1111111111111116
The path is:  [1, 3, 4]
numbers after decimal point:  16


In [204]:
G.edges(data='speed')
#G.edges(data='weight')
#G.edges(data=True)

OutMultiEdgeDataView([(1, 2, 18), (1, 3, 15), (2, 4, 10), (3, 4, 7), (4, 5, 5)])

In [186]:
new_graph.edges(data='speed')
#new_graph.edges(data='weight')
#new_graph.edges(data=True)

OutMultiEdgeDataView([(1, 2, 18.0), (1, 3, 15.0), (2, 4, 10.0), (2, 1, 1.0), (3, 4, 7.0), (3, 1, 1.0), (4, 5, 5.0), (4, 2, 1.0), (4, 3, 1.0), (5, 4, 1.0)])